In [18]:
import pandas as pd

In [19]:
df=pd.read_csv('SDI_100.csv')
df

,location,A_max
0,Afghanistan,-1326.483
1,Albania,-644.061
2,Algeria,-702.612
3,American Samoa,-581.472
4,Andorra,-214.014
...,...,...
198,Vietnam,-773.277
199,Virgin Islands,-405.819
200,Yemen,-1184.566
201,Zambia,-999.405


In [20]:
df['year']=2020

In [21]:
new_df = pd.DataFrame()

for _, row in df.iterrows():
    for year in range(2021, 2101, 1):
        new_row = row.copy()
        new_row['year'] = year
        new_df = new_df.append(new_row, ignore_index=True)

df_2 = pd.concat([df, new_df])

In [22]:
df_2

,location,A_max,year
0,Afghanistan,-1326.483,2020.0
1,Albania,-644.061,2020.0
2,Algeria,-702.612,2020.0
3,American Samoa,-581.472,2020.0
4,Andorra,-214.014,2020.0
...,...,...,...
16235,Zimbabwe,-1057.956,2096.0
16236,Zimbabwe,-1057.956,2097.0
16237,Zimbabwe,-1057.956,2098.0
16238,Zimbabwe,-1057.956,2099.0


In [23]:
# df_2['sdi_pred']=df_2.apply(lambda row:1+(row['A_avg']/row['year']), axis=1)

# testing A_min to avoid "underestimating" the As at the beginning of the prediction period
df_2['sdi_pred'] = df_2.apply(lambda row:1+(row['A_max']/row['year']), axis=1)

In [24]:
df_2['sdi_cluster'] = df_2.apply(lambda row: 'Low' if row['sdi_pred']< 0.454743 
                                else 'Low-middle' if row['sdi_pred'] < 0.607679
                                else 'Middle'if row['sdi_pred']< 0.805129
                                else 'High', axis=1)

In [25]:
# #TODO: do it in SQL before

# df_2 = df_2[df_2['country'].isin(nations)]
# df_2

In [26]:
# checks 

df_2[df_2['sdi_pred'] == min(df_2['sdi_pred'])]

,location,A_max,year,sdi_pred,sdi_cluster
164,Somalia,-1855.461,2020.0,0.081455,Low


In [27]:
df_2[df_2['sdi_pred'] == max(df_2['sdi_pred'])]

,location,A_max,year,sdi_pred,sdi_cluster
13919,Switzerland,-143.349,2100.0,0.931739,High


In [28]:
lst = ["Tobacco", "Alcohol use", "Metabolic risks", "Dietary risks", "Low physical activity"]

# Convert list to DataFrame
df_list = pd.DataFrame(lst, columns=['Risk_Factor'])

# Create a temporary key for merging
df_2['key'] = 0
df_list['key'] = 0

# Perform the cross join
df_3 = pd.merge(df_2, df_list, how='outer', on='key')

df_3

,location,A_max,year,sdi_pred,sdi_cluster,key,Risk_Factor
0,Afghanistan,-1326.483,2020.0,0.343325,Low,0,Tobacco
1,Afghanistan,-1326.483,2020.0,0.343325,Low,0,Alcohol use
2,Afghanistan,-1326.483,2020.0,0.343325,Low,0,Metabolic risks
3,Afghanistan,-1326.483,2020.0,0.343325,Low,0,Dietary risks
4,Afghanistan,-1326.483,2020.0,0.343325,Low,0,Low physical activity
...,...,...,...,...,...,...,...
82210,Zimbabwe,-1057.956,2100.0,0.496211,Low-middle,0,Tobacco
82211,Zimbabwe,-1057.956,2100.0,0.496211,Low-middle,0,Alcohol use
82212,Zimbabwe,-1057.956,2100.0,0.496211,Low-middle,0,Metabolic risks
82213,Zimbabwe,-1057.956,2100.0,0.496211,Low-middle,0,Dietary risks


In [29]:
# Drop the key and A_max columns
df_3.drop(columns=['key', 'A_max'], inplace=True)

# rename sdi_pred to sdi, and Risk_Factor to risk
df_3.rename(columns={'sdi_pred':'SDI', 'Risk_Factor':'Risk', 'sdi_cluster': 'cluster_sdi'}, inplace=True)

df_3

,location,year,SDI,cluster_sdi,Risk
0,Afghanistan,2020.0,0.343325,Low,Tobacco
1,Afghanistan,2020.0,0.343325,Low,Alcohol use
2,Afghanistan,2020.0,0.343325,Low,Metabolic risks
3,Afghanistan,2020.0,0.343325,Low,Dietary risks
4,Afghanistan,2020.0,0.343325,Low,Low physical activity
...,...,...,...,...,...
82210,Zimbabwe,2100.0,0.496211,Low-middle,Tobacco
82211,Zimbabwe,2100.0,0.496211,Low-middle,Alcohol use
82212,Zimbabwe,2100.0,0.496211,Low-middle,Metabolic risks
82213,Zimbabwe,2100.0,0.496211,Low-middle,Dietary risks


In [30]:
df_3.columns

Index(['location', 'year', 'SDI', 'cluster_sdi', 'Risk'], dtype='object')

In [31]:
# # Define the new order of columns


# new_order = ['sdi_cluster', 'country', 'year', 'sdi', 'risk', 'ASR', 'ASR_predicted']

# # Rearrange the columns
# df_3 = df_3[new_order]

In [32]:
df_3.to_csv('SDI_pred.csv', index=False)